In [9]:
from os import environ, path, walk
from pocketsphinx import *
from sphinxbase import *
import fnmatch
relative_path = "../ps_data"

In [10]:
def create_decoder_digits(dic, grammar, rule):
    """Create a decoder based on the goforward custom grammar"""
    config = Decoder.default_config()
    config.set_string('-hmm', relative_path +'/model/en-us')  # acoustic model
    config.set_string('-dict', relative_path +'/lex/' + dic)  # lexicon / dictionary
    decoder_gofwd = Decoder(config)

    # Now we use a custom language model
    # Prepare the grammar to be used
    jsgf = Jsgf(relative_path +'/jsgf/' + grammar)  # load the grammar file
    rule = jsgf.get_rule('digits.' + rule)  # choose the rule
    fsg = jsgf.build_fsg(rule, decoder_gofwd.get_logmath(), 7.5)  # build the grammar rule
    fsg.writefile('../outputs/' + grammar)  # write the compiled grammar rule as an external file

    # Now set the fsg grammar rule in the decoder
    decoder_gofwd.set_fsg("../outputs/" + grammar, fsg)  # load the pre-recorded compiled grammar rule in the decoder
    decoder_gofwd.set_search("../outputs/" + grammar)  # and set it as the grammar to use

    return decoder_gofwd

In [11]:
def rundecoder(file_path, decoder):
    # Start the decoder
    decoder.start_utt()

    # Open the file to decode
    stream = open(file_path, 'rb')
    uttbuf = stream.read(-1)

    # Process the file with the decoder
    if uttbuf:
        decoder.process_raw(uttbuf, False, True)
    else:
        print("Error reading speech data")
        exit()
    decoder.end_utt()

    # test for empty hypothesis and replace the output with an empty string if needed
    if decoder.hyp() is None:
        best_hypothesis = ''
    else:
        best_hypothesis = decoder.hyp().hypstr

    # Print the results
    print('Best hypothesis: ', best_hypothesis,
        "\n model score: ", decoder.hyp().best_score,
        "\n confidence: ", decoder.get_logmath().exp(decoder.hyp().prob))

    print('Best hypothesis segments: ', [seg.word for seg in decoder.seg()])

    # Access N best decodings
    N = 8
    print('Best ' + str(N) + ' hypothesis: ')
    for best, i in zip(decoder.nbest(), range(N)):
        print(best.hypstr, best.score)


In [13]:
# Define the path of the file to process
file_path = relative_path +'/example/ex_digits.raw'

# Instantiate the decoder
# decoder = create_decoder_ngram()  # use the N-gram language model
decoder = create_decoder_digits("digits.dict", "digits.jsgf", "rule3")
rundecoder(file_path, decoder)

Best hypothesis:  zero one eight 
 model score:  -3251 
 confidence:  1.0
Best hypothesis segments:  ['<sil>', 'zero(4)', '(NULL)', 'one', '(NULL)', 'eight', '(NULL)', '<sil>']
Best 8 hypothesis: 
four one eight -2999
four one eight -2999
four one eight -2999
zero one eight -3032
zero one eight -3032
zero one eight -3032
eight zero one eight -3154
eight zero one eight -3154


INFO: pocketsphinx.c(151): Parsed model-specific feature parameters from ../ps_data/model/en-us/feat.params
Current configuration:
[NAME]			[DEFLT]		[VALUE]
-agc			none		none
-agcthresh		2.0		2.000000e+00
-allphone				
-allphone_ci		yes		yes
-alpha			0.97		9.700000e-01
-ascale			20.0		2.000000e+01
-aw			1		1
-backtrace		no		no
-beam			1e-48		1.000000e-48
-bestpath		yes		yes
-bestpathlw		9.5		9.500000e+00
-ceplen			13		13
-cmn			live		batch
-cmninit		40,3,-1		41.00,-5.29,-0.12,5.09,2.48,-4.07,-1.37,-1.78,-5.08,-2.05,-6.45,-1.42,1.17
-compallsen		no		no
-dict					../ps_data/lex/digits.dict
-dictcase		no		no
-dither			no		no
-doublebw		no		no
-ds			1		1
-fdict					
-feat			1s_c_d_dd	1s_c_d_dd
-featparams				
-fillprob		1e-8		1.000000e-08
-frate			100		100
-fsg					
-fsgusealtpron		yes		yes
-fsgusefiller		yes		yes
-fwdflat		yes		yes
-fwdflatbeam		1e-64		1.000000e-64
-fwdflatefwid		4		4
-fwdflatlw		8.5		8.500000e+00
-fwdflatsfwin		25		25
-fwdflatwbeam		7e-29		7.000000e-29
-fwdtree		yes		yes